In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def xavier_init(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1./tf.sqrt(in_shape[0]/2.))
  return val

In [ ]:
Disc_W = {"disc_H" : tf.Variable(xavier_init([image_dimension, D_H1])),
          "disc_final" : tf.Variable(xavier_init([D_H1, 1]))
          }

Disc_Bias = {"disc_H" : tf.Variable(xavier_init([image_dimension])),
          "disc_final" : tf.Variable(xavier_init([1]))
          }
         

In [ ]:
Gen_W = {"Gen_H" : tf.Variable(xavier_init([Z_dim + C_dim, G_H1])),
          "Gen_final" : tf.Variable(xavier_init([G_H1, image_dimension]))
          }

Gen_Bias = {"Gen_H" : tf.Variable(xavier_init([G_H1])),
          "Gen_final" : tf.Variable(xavier_init([image_dimension]))
          }
         

In [ ]:
Q_W = {"Q_H" : tf.Variable(xavier_init([image_dimension, Q_H1])),
          "Q_final" : tf.Variable(xavier_init([Q_H1, C_dim]))
          }

Q_Bias = {"Q_H" : tf.Variable(xavier_init([Q_H1])),
          "Q_final" : tf.Variable(xavier_init([C_dim]))
          }
         

In [ ]:
X = tf.placeholder(tf.float32, shape = [None, image_dimension])
Z = tf.placeholder(tf.float32, shape = [None, Z_dim])
C = tf.placeholder(tf.float32, shape = [None, C_dim])

In [ ]:
def Z_distribution(m, n):
  return np.random.uniform(-1., 1., size = [m, n])

In [ ]:
def C_distribution(m, C_dim):
  return np.random.multinomial(1, C_dim * [1/C_dim], size = m) #10 * [1/10] - 10 values of prob 0.1 each

In [ ]:
def Generator(z, c):
  input = tf.concat(axis = 1, values = [z, c])
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Gen_W['Gen_H']), Gen_Bias['Gen_H']))
  gen_output = tf.add(tf.matmul(G_hidden_layer, Gen_W['Gen_final']), Gen_Bias['Gen_final'])
  gen_prob_output = tf.nn.sigmoid(gen_output)
  return gen_prob_output

In [ ]:
def Discriminator(x):
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Disc_W['disc_H']), Disc_Bias['disc_H']))
  disc_output = tf.add(tf.matmul(D_hidden_layer, Disc_W['disc_final']), Disc_Bias['disc_final'])
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output  

In [ ]:
def Auxiliary_NN(x):
  Q_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Q_W['Q_H']), Q_Bias['Q_H']))
  Q_output = tf.add(tf.matmul(Q_hidden_layer, Q_W['Q_final']), Q_Bias['Q_final'])
  Q_prob_output = tf.nn.softmax(Q_output)
  return Q_prob_output  

In [ ]:
#Building the network

samples_generated = Generator(Z, C)
real_output_disc = Discriminator(X)
fake_output_disc = Discriminator(samples_generated)
estimated_c = Auxiliary_NN(samples_generated)

In [ ]:
Disc_Loss = -tf.reduce_mean(tf.log(real_output_disc + 1e-7) + tf.log(1.0 - fake_output_disc + 1e-7))
Gen_Loss = -tf.reduce_mean(tf.log(fake_output_disc + 1e-7))
Q_Loss = -tf.reduce_mean(tf.reduce_sum(tf.log(estimated_c + 1e-7)*C, 1)) 

In [ ]:
Disc_params = [Disc_W['disc_H'], Disc_W['disc_final'], Disc_Bias['disc_H'], Disc_Bias['disc_final']]
Gen_params = [Gen_W['Gen_H'], Gen_W['Gen_final'], Gen_Bias['Gen_H'], Gen_Bias['Gen_final']]
Q_params = [Q_W['Q_H'], Q_W['Q_final'], Q_Bias['Q_H'], Q_Bias['Q_final']]

In [ ]:
gen_optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Gen_Loss, var_list = Gen_params)
Disc_optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Disc_Loss, var_list = Disc_params)
Aux_optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Q_Loss, var_list = Gen_params + Q_params)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  X_batch, _ = mnist.train.next_batch(batch_size)
  z_noise = Z_distribution(batch_size, Z_dim)
  c_noise = C_distribution(batch_size, C_dim)

  _, Disc_loss_epoch = sess.run([Disc_optimize, Disc_Loss], feed_dict = {X:X_batch, C:c_noise})
  _, Gen_loss_epoch = sess.run([gen_optimize, Gen_Loss], feed_dict = {Z:z_noise, C:c_noise})
  _, Aux_loss_epoch = sess.run([Aux_optimize, Q_Loss], feed_dict = {Z:z_noise, C:c_noise})

  if epoch%2000 == 0:
    print('Steps: {0}, Disc Loss: {1}, Generator Loss: {2}, Aux Loss: {3}'.format(epoch, Disc_loss_epoch, Gen_loss_epoch, Aux_loss_epoch))

In [ ]:
test_output , _ = Generator(Z, C)
n = 6
canvas = np.empty((28*n, 28*n))

for i in range(n):
  Z_noise = Z_distribution(batch_size, Z_dim)
  C_noise = C_distribution(batch_size, C_dim)


  #Another way to pass the value  of C_noise
  id = 5
  C_noise = np.zeros((batch_size, C_dim))
  C_noise[range(batch_size), id] = 1  #encode the vector such that along the id index for all samples the val is 1
  #when this is done acc to each id there is a particular style feature coded into "c" and that style is the conditioning for the vals generated


  generated_image  = sess.run(test_output, feed_dict = {Z:Z_noise, C:C_noise})
  for j in range(n):
    canvas[i * 28: (i+1) * 28, j * 28: (j+1) * 28] = generated_image[j].reshape([28, 28])


plt.figure(figsize = (n, n))
plt.imshow(canvas, origin = "upper", cmap = 'gray')
plt.show()